# Merging and Combining data  

Sometimes we need to combine data. In many instances, adding additional data is a way to enrich both the questions that can be asked of your original data set and the answers that can be provided. This notebook looks at ways to  create one dataframe from two or more dataframes.  That's colloquially known as a **merge** or a **join**. There are lots of ways to do this.  We do a couple but supply references to more at the end.  

Along the way we take an extended detour to review methods for **downloading** and **unzipping** compressed files.  The tools we use here have a broad range of other applications, including web scraping.  

Outline:  

* [MovieLens data](#movielens).  A collection of movies and individual ratings.  
* [Automate file download](#requests).  Use the requests package to get a zipped file, then other tools to unzip it and read in the contents.  
* [Merge movie names and ratings](#merge-movies).  Merge information from two dataframes with Pandas' `merge` function.  

**Note: requires internet access to run.**  

In [ ]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 

# these are new 
import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import shutil                   # file management tools 
import os                       # operating system tools (check files)

## A Quick Primer on Merging

First, lets look at some simple examples and see how merging works. Then we will explore this more with the Movie data. 

There are two key commands here. First here is `pd.append`. This operation is useful when we just want to stack, say, two DataFrames on top of each other. For example, in the basketball dataset, we could layer on the 2017 and 2018 data. 

The next important command here is ``pd.merge()`` which is a pandas method applied to two data frames. What you need to provide pandas are the two data frames, the type of merge, and the ``key`` to merge on. Below, I'm going to look at a couple of common examples that I come across, one-to-one merges and many to one merges (in the documentation there is a discussion of many to many merges). Lets see this:

### Append

This operation is fairly straigt foward, lets just stack on ontop of each other. 

In [ ]:
gdp_rich_data = {"cntry": ["USA", "JPN"],
               "gdp": [100, 110]}

gdp_poor_data = {"cntry": ["TZA", "SAR"],
               "gdp": [25, 50]}

df_one = pd.DataFrame(gdp_rich_data)
df_two = pd.DataFrame(gdp_poor_data)

df_one.append(df_two, ignore_index = True)


In [ ]:
gdp_rich_data = {"cntry": ["USA", "JPN"],
               "gdp": [100, 110],
                "cpi": [2, 3]}

gdp_poor_data = {"cntry": ["TZA", "SAR"],
               "gdp_f": [25, 50]}

df_one = pd.DataFrame(gdp_rich_data)
df_two = pd.DataFrame(gdp_poor_data)

df_one.append(df_two, ignore_index = True)


### Merge

This is the key operation to combine two datasets on a column (as opposed to pd.append adds rows). Lets see how this works in a simple **one-to-one** merge:

In [ ]:
gdp_data = {"cntry": ["USA", "JPN"],
               "gdp": [100, 110]}

cpi_data = {"cntry": ["USA", "JPN"],
               "cpi": [10, 15]}

df_one = pd.DataFrame(gdp_data)
df_two = pd.DataFrame(cpi_data)

# Here is a classic one-to-one merge
combo = pd.merge(df_one, df_two,# left df, right df
                 on='cntry',       # link with cntry
                 how='left',        # add to left...does this matter here?
                 indicator=True)  # Tells us what happend

combo.head()

So we have two data frames, one with gdp data and one with cpi data for two countries in **both** dataframes. We want to create one dataframe out of the two initial ones. So then ``pd.merge`` takes the left data frame, the right dataframe, and then we provide some options. 
- The most important one is ``on`` this provides the key to match up the two dataframes. In this case this is the country.
- The second important on is ``how`` this tells us the method to which the merge is undertaken. There are four options, ``left`` (which just takes the keys from the left), ``right`` (which just takes the keys on the right), ``inner`` (which just takes the intersection of the keys), and ``outer`` (which takes the union). Note here it does not matter because there is an exact one-to-one correspondence of the keys.
- Finally, I like to use the indicator option that will generate a new column with a description of what happened. In this case, the key was found on both.

Below is another example. It is still a one-to-one merge, but now there on some keys on one dataframe that are not in another...

In [ ]:
# Here is still a one-to-one merge, but a bit more complicated...
# Since the two dataframes have different stuff missing...

gdp_data = {"cntry": ["USA", "JPN", "BRZ"],
               "gdp": [100, 110, 50]}
cpi_data = {"cntry": ["USA", "JPN", "CHN"],
               "cpi": [10, 15, 8]}
           #"gdp": [100, 110, 75]}

df_one = pd.DataFrame(gdp_data)
df_two = pd.DataFrame(cpi_data)

# Here is a classic one-to-one merge
combo = pd.merge(df_one, df_two,   # left df, right df
                 how='left',      # Try the different options, inner, outer, left, right...what happens.
                 on='cntry',       # link with cntry
                 indicator=True)  # Tells us what happend

combo.head()

**Exercise: Try different ``how`` operations...what happens?** 

---

Now we can also merge on combinations of values. In the example below, it would be country by time merge...

In [ ]:
gdp_data = {"cntry": ["USA", "JPN","USA", "JPN"],
               "gdp": [100, 110, 95, 105],
           "year": [2017, 2017 , 2016, 2016] }

cpi_data = {"cntry": ["USA", "JPN","USA", "JPN"],
               "cpi": [10, 15, 9, 14],
            "year": [2017, 2017 , 2016, 2016] }

df_one = pd.DataFrame(gdp_data)
df_two = pd.DataFrame(cpi_data)

# Here is a classic one-to-one merge
combo = pd.merge(df_one, df_two,# left df, right df
                 on=['cntry', "year"],       # link with cntry
                 how='left',        # add to left...does this matter here?
                 indicator=True)  # Tells us what happend

combo.head(10)

What if you set the `on` option to be just `cntry`. What would happen. Do you see what is going on here?

---

Now below is an example of a **many-to-one merge**. That is we will assign many values to one depending upon the key. 

In [ ]:
gdp_data = {"cntry": ["USA", "JPN", "BRZ"],
               "gdp": [100, 110, 50]}

state_pop_data = {"cntry": ["USA", "USA", "JPN", "JPN"],
                  "state": ["NY", "AK", "Kyoto", "Tokyo"],
                  "pop": [10, 5, 8, 4]}

df_one = pd.DataFrame(gdp_data)
df_two = pd.DataFrame(state_pop_data)

# Here is a classic one-to-one merge
combo = pd.merge(df_one, df_two,   # left df, right df
                 how='inner',      # Try the different options, inner, outer, left, right...what happens.
                 on='cntry',       # link with cntry
                 indicator=True)  # Tells us what happend

combo

Note what is going on here. For each state (or prefecture in Japan)....(the many part) the countries GDP is assigned to the one value associated with the key, in this case GDP. Again, **Try different ``how`` operations ...what happens?** 

Lets see this at work for a lager, more interesting dataset.

## [MovieLens data](https://grouplens.org/datasets/movielens/) 

The data comes as a zip file that contains several csv's.  We get the details from the README inside.  (It's written in Markdown, so it's easier to read if we use a browser to format it.  Or we could cut and paste into a Markdown cell in an IPython notebook.)  

The file descriptions are:  

* `ratings.csv`:  each line is an individual film rating with the rater and movie id's and the rating.  Order:  `userId, movieId, rating, timestamp`. 
* `tags.csv`:  each line is a tag on a specific film.  Order:  `userId, movieId, tag, timestamp`. 
* `movies.csv`:  each line is a movie name, its id, and its genre.  Order:  `movieId, title, genres`.  Multiple genres are separated by "pipes" `|`.   
* `links.csv`:  each line contains the movie id and corresponding id's at [IMBd](http://www.imdb.com/) and [TMDb](https://www.themoviedb.org/).  

The easy way to input this data is to download the zip file onto our computer, unzip it, and read the individual csv files using `read.csv()`.  But **anyone can do it the easy way**.  We want to automate this, so we can redo it without any manual steps.  This takes some effort, but once we have it down we can apply it to lots of other data sources.  

### Automate file download 

We're looking for an automated way, so that if we do this again, possibly with updated data, the whole process is in our code.  Automated data entry involves these steps: 

* Get the file.  We use the [requests](http://docs.python-requests.org/) package, which handles internet files and comes pre-installed with Anaconda. This kind of thing was hidden behind the scenes in the Pandas `read_csv` function, but here we need to do it for ourselves. The package authors add:  
>Recreational use of other HTTP libraries may result in dangerous side-effects, including: security vulnerabilities, verbose code, reinventing the wheel, constantly reading documentation, depression, headaches, or even death.
* Convert to zip.   Requests simply loads whatever's at the given url. The [io](https://docs.python.org/3.5/library/io.html) module's `io.Bytes` reconstructs it as a file, here a zip file.  
* Unzip the file.  We use the [zipfile](https://docs.python.org/3.5/library/zipfile.html) module, which is part of core Python, to extract the files inside.   
* Read in the csv's.  Now that we've extracted the csv files, we use `read_csv` as usual.  

We found this [Stack Overflow exchange](http://stackoverflow.com/questions/23419322/download-a-zip-file-and-extract-it-in-memory-using-python3) helpful. 

**Digression.**  This is probably more than you want to know, but it's a reminder of what goes on behind the scenes when we apply `read_csv` to a url.  Here we grab whatever is at the url.  Then we get its contents, convert it to bytes, identify it as a zip file, and read its components using `read_csv`.  It's a lot easier when this happens automatically, but a reminder what's involved if we ever have to look into the details.  

In [ ]:
# get "response" from url 
url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
r = requests.get(url) 

# describe response 
print('Response status code:', r.status_code)
# Here is a link to what different coes mean: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status
# Quick summary 200 is good, 400 is bad, i.e. its not responsive may not exist. Othere indicate something
# more complicated might be going on.

print('Response type:', type(r))
print('Response .content:', type(r.content)) 
print('Response headers:\n', r.headers, sep='')

In [ ]:
# convert bytes to zip file  
mlz = zf.ZipFile(io.BytesIO(r.content)) 
print('Type of zipfile object:', type(mlz))

In [ ]:
# what's in the zip file?
# This is really cool...
mlz.namelist()

In [ ]:
# extract and read csv's
movies  = pd.read_csv(mlz.open(mlz.namelist()[2]))
ratings = pd.read_csv(mlz.open(mlz.namelist()[3]))

tags = pd.read_csv(mlz.open(mlz.namelist()[5]))

In [ ]:
tags.head(100)

In [ ]:
# Whip through the dataframes and look what is going on...

for df in [movies, ratings]:
    print('Type:', type(df))
    print('Dimensions:', df.shape)
    print('Variables:', list(df))
    print('First few rows', df.head(3), '\n')

### Merging ratings and movie titles 

The movie ratings in the dataframe `ratings` give us individual opinions about movies, but they don't include the name of the movie.   Why not?  Rather than include the name every time a movie is rated, the MovieLens data associates each rating with a movie code, than stores the names of movies associated with each movie code in the dataframe `movies`.  We run across this a lot:  some information is in one data table, other information is in another. Here, since this was an intientional decision to split up the files, there is a natural ``key`` that can be used to link the two. 

Our **want** is therefore to add the movie name to the `ratings` dataframe.  We say we **merge** the two dataferames.  There are lots of ways to merge.  Here we do one as an illustration.  

### Merging

Here's roughly what's involved in what we're doing.  We take the `movieId` variable from `ratings` and look it up in `movies`.  When we find it, we look up the `title` and add it as a column in `ratings`.  The variable `movieId` is common, so we can use it to link the two dataframes.  **Is this a many-to-one merge or one-to-one merge?**

In [ ]:
print('Dimensions of ratings:', ratings.shape)
print('Dimensions of movies:', movies.shape)

# Again note how there are more ratings than movies...
# It is a many-to-one merge, assign the many ratings
# to a ONE movie title. 

combo = pd.merge(ratings, movies,   # left and right df's
                 how='left',        # add to left 
                 on='movieId',      # link with this variable/column 
                 indicator=True) 

print('Dimensions of new df:', combo.shape)

combo.head()

---

**Exercise**: 

- Merge the tags to our new DataFrame. Which movie is the "sandra 'boring' bullock" one?

- Now try the original merge (ratings and movies) with right part...What is going on here?

---

In [ ]:
combo = pd.merge(ratings, movies,   # left and right df's
                 how='right',        # add to left 
                 on='movieId',       # link with this variable/column 
                 indicator=True) 

print('Dimensions of new df:', combo.shape)

combo.head()

combo[combo["_merge"] == "right_only"]

---

**Practice with grouping, slicing, visualization**

Some of these we know how to do, the others we don't.  For the ones we know, what is the answer?  For the others, what (in loose terms) do we need to be able to do to come up with an answer?  

* What is the overall average rating?  By genre?
* What is the overall distribution of ratings?  


In [ ]:
print("Average Rating",round(combo.rating.mean(),2))
print("Median Rating",round(combo.rating.median(),2))

fig, ax = plt.subplots()

combo.rating.plot(ax = ax, kind = "hist", bins = 15)
plt.show()

Lets ask a couple of more questions, this practices `groupby`
* What is the average rating of each movie?  
* How many ratings does each movie get? 

In [ ]:
grouped = combo.groupby("title")
# This is similar syntax to that used in the Chiplotle data
# Create the groupby object.

movie_ratings = grouped.rating.agg(["count","mean"])
# Then aggregate the ratings...how? In two ways, we pass a list
# saying, take the count, take the mean.

movie_ratings.head(20)

In [ ]:
# Now lets look at movies, with only at least 20 ratings, and then sort them.
# So what are the best rated movies...

movie_ratings[movie_ratings["count"] >= 20.0]["mean"].sort_values().tail(20)

Then the final thing we could do is create a scatter plot of average rating versus count, i.e. do better rated movies have more ratings?

In [ ]:
fix, ax = plt.subplots()

ax.scatter(movie_ratings[movie_ratings["count"] >= 50.0]["mean"], 
           movie_ratings[movie_ratings["count"] >= 50.0]["count"], alpha = 0.50)

plt.show()

**Exercise:** Rather than grouping by movie, you could group by the rater... are there some raters that are systematically negative, systematically positive? Even more interesting question, note that there is a time dimension, so maybe there is a time effect or life cycle component? 